In [9]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

In [10]:
awg.awg._ch[1].sync_marker_high=1.5

0.5

In [48]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg =awg
seq.start_new_segment('Starting segment')


seq.asc(length_mus = 10, green=True, orange = True)


seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000wait    1       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [49]:
seq.write_seq()

MCAS 'Laser' has been generated.


In [22]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0350 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0220 seconds


In [23]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0160 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0160 seconds


## Ramsey

In [4]:
[1,400e-3]
0.5,200e-3

(0.5, 0.2)

In [11]:
AWG_M8190A_qudi.awg._ch[1].output_amplitude

0.5

In [7]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

def awg_ramsey(taus):
    seq = mcas.MultiChSeq('hahn_echo', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    awg_freq = 2206.54 # in MHz
    electron_amp = 0.2 # AU 300mV peak to peak
    electron_pi = 1.352/2 # us
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
    for tau in taus:
        for phase in phases:

            seq.start_new_segment('hahn')
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                    length_mus=electron_pi / 2.)
            seq = awg_AddWait(seq, tau)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],
                                phases=[phase]),
                    length_mus=electron_pi / 2.)
            seq.asc(length_mus=0.1)
            
#             seq = AddRfSafety(seq)
            seq = awg_AddReadout(seq)
    return seq

def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 1000
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)
    
    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq

def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq

# taus = np.linspace(100,1000,91)
# taus = np.arange(0.01,130,1.)
taus = np.arange(0.01, 5, 0.03)


seqFID = awg_ramsey(taus)
seqFID.awg = awg




# pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
#                                            controlled_variable= taus,
#                                            number_of_lasers=2*len(taus), 
#                                            laser_ignore_list=[], 
#                                            alternating=True, 
#                                            units=('s', 'arb. u.'))




In [8]:

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= taus*1e-6,
                                           number_of_lasers=2*len(taus), 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('s', 'arb. u.'))

In [9]:
len(taus)

167

In [10]:
# seqEcho.sequences['awg'][1].print_info()

In [11]:
seqFID.write_seq()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0120 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0120 seconds
MCAS 'hahn_echo' has been generated.


In [12]:
seqFID.run_sequence()

writing sequence hahn_echo on AWG_M8190A_qudi ch 1 took 3.0985 seconds
writing sequence hahn_echo on AWG_M8190A_qudi ch 2 took 2.9476 seconds


In [13]:
seqFID.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0120 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0110 seconds


33.333333333333336